# 인터프리터에서 변수, 조건문 다루기

덧셈만 하던 언어에 이름(변수)와 조건문을 추가해 보자. 더많이 추가해 보고 싶으면 나머지 사칙연산을 나름대로 추가해 보아도 된다.

In [1]:
data Expr = Var String         -- x
          | Val Int            -- n
          | Add Expr Expr      -- e1 + e2
          -- | Sub Expr Expr      -- e1 - e2
          -- | Mul Expr Expr      -- e1 * e2
          -- | Div Expr Expr      -- e1 / e2
          | If Expr Expr Expr  -- if e then e1 else e0
          deriving Show

-- 실행시 이름에 대응되는 값을 찾아볼 수 있는 (실행)환경(environment)
type Env = [ (String, Int) ]  

eval :: Env -> Expr -> Int
eval env (Var x) = case lookup x env of
                     Nothing -> error (x ++ " not found") 
                     Just n  -> n
eval env (Val n) = n
eval env (Add e1 e2) = eval env e1 + eval env e2
eval env (If e e1 e0) = case eval env e of
                          0 -> eval env e0
                          _ -> eval env e1

`lookup`이라는 함수는 찾으면 데이터 생성자 `Just`로 포장된 값을, 없으면 데이터 상수 `Nothing`을 돌려준다.
참고로 `Just`와 `Nothing`은 아래와 같이 표준라이브러리에 정의되어 있는 `Maybe` 타입의 데이타 생성자 및 상수이다.
```haskell
data Maybe a = Nothing | Just a
```

In [2]:
[1,1,2,2] ++ [3,3,4,4]

[1,1,2,2,3,3,4,4]

In [18]:
:type lookup
lookup "x" []
lookup "x" [("x",3),("y",4)]
lookup "y" [("x",3),("y",4)]
lookup "z" [("x",3),("y",4)]

lookup :: forall a b. Eq a => a -> [(a, b)] -> Maybe b

Nothing

Just 3

Just 4

Nothing

In [19]:
mylookup y []         = Nothing
mylookup y ((x,v):ps) = if x==y then Just v
                                else mylookup y ps
mylookup "x" []
mylookup "x" [("x",3),("y",4)]
mylookup "y" [("x",3),("y",4)]
mylookup "z" [("x",3),("y",4)]

Nothing

Just 3

Just 4

Nothing

우리 장난감 언어의 추상 문법 구조와 인터프리터를 정의한
위 하스켈 코드를 프로그래밍언어 관련 교과서나 논문 및 기술문서 등에서
많이 볼 수 있는 수식 기호 표기로 옮겨놓자면 다음과 같다.

###### 문법 syntax
$ x,y,z,\ldots \in \textit{String} $

$ n,m \in \textit{Int} $

$ e \in \textit{Expr} ::= x ~\mid~ n ~\mid~ e_1+e_2 ~\mid~ \texttt{if}\,~e~\,\texttt{else}\,~e_1~\,\texttt{else}\,~e_0 $

###### 환경 environment
$ \textit{Env} = \textit{String} \xrightarrow{\textrm{fin}} \textit{Int} \\
  \sigma \in \textit{Env} ::= \{ x_1 \mapsto n_1,~\ldots,~x_k\mapsto n_k \}$

###### 실행규칙 evaluation rules

$\displaystyle
\frac{~~}{
      x \Downarrow_\sigma \sigma(x)}
\qquad
\frac{~~}{
      n \Downarrow_\sigma n}
$

$~$

$\displaystyle
\frac{e_1 \Downarrow_\sigma n_1 \quad e_2 \Downarrow_\sigma n_2 \quad n = n_1+n_2}{
      e_1 + e_2 \Downarrow_\sigma n}
$

$~$

$\displaystyle
\frac{e\Downarrow_\sigma 0 \quad e_1 \Downarrow_\sigma n}{
      \texttt{if}\,~e~\,\texttt{else}\,~e_1~\,\texttt{else}\,~e_0 \Downarrow_\sigma n}
\qquad
\frac{e\Downarrow_\sigma m \quad e_1 \Downarrow_\sigma n \quad m\neq0}{
      \texttt{if}\,~e~\,\texttt{else}\,~e_1~\,\texttt{else}\,~e_0 \Downarrow_\sigma n}
$

----

`if b then (x + 3) else y`와 같은 코드를 `Expr` 타입의 하스켈 데이터로 다음과 같이 나타낼 수 있다.

In [4]:
e = If (Var "b") (Add (Var "x") (Val 3)) (Var "y")

이 코드를 실행하려면 $\sigma = \{ x \mapsto 2, y \mapsto 4, b \mapsto 0 \}$과 같은 환경이 필요하다.
인터프리터 함수에 환경과 식을 같이 넘겨 ($\texttt{eval}~\,\sigma\,~\texttt{e}$) 실행하면 된다. 

In [5]:
eval [("x",2), ("y",4), ("b",0)] e

4

이번에는 $\{ x \mapsto 2, y \mapsto 4, b \mapsto 0 \}$ 환경으로 실행해 보자.

In [23]:
eval [("x",2), ("y",4), ("b",1)] e  

5

실행 환경은 코드에 나타나는 이름에 대한 값을 모두 포함하고 있어야 한다.
그렇지 않을 경우 예컨대 $\{ x \mapsto 2, y \mapsto 4 \}$ 같은 환경으로 실행한다면 ...

In [7]:
eval [("x",2), ("y",4)]          e -- 에러 발생

: 

## HW1-compiler2019
제출기한: 2019-09-19 목요일 밤 11:59까지

점수: 5점. 하지만 이 점수는 임시로 받아 둔 점수일 뿐이고 중간고사에 관련 문제가 출제되어 그 문제를 제대로 해결한 정도에 따라 과제에서 받은 점수의 일정 비율만 실제 과제 점수로 책정된다. 과제는 맞게 제출했으나 중간고사에 관련 문제를 전혀 풀지 못할 정도로 실제 이해하지 못하고 있는 경우는 음의 비율 즉 중간고사 결과에 따라서는 과제 점수가 마이너스 점수로 책정될 수도 있다.

 * 이름: ???
 * 학번: 201?????

아래 `vars :: Expr -> [String]` 함수는 주어진 식에 나타난 변수들을 모두 모은 리스트를 돌려준다.
이 함수를 이용해 어떤 환경이 주어진 식에 나타난 변수를 모두 정의하고 있는지 검사하는 `closedBy :: Expr -> Env -> Bool`를 작성하라.

In [8]:
import Data.List (union)

vars :: Expr -> [String]
vars (Var x) = [x]
vars (Val _) = []
vars (Add e1 e2) = vars e1 `union` vars e2
vars (If e e1 e0) = vars e `union` vars e1 `union` vars e0

In [9]:
e = If (Var "b") (Add (Var "x") (Val 3)) (Var "y")

vars e

["b","x","y"]

In [38]:
closedBy :: Expr -> Env -> Bool
closedBy e env = undefined

In [37]:
e = If (Var "b") (Add (Var "x") (Val 3)) (Var "y")

closedBy e [("x",2), ("y",4), ("z",6), ("b",0)] -- True
closedBy e [("x",2), ("y",4), ("b",0)]          -- True
closedBy e [("x",2), ("y",4)]                   -- False

: 

아래는 과제에 참고가 될 수도 있는 내용이다.

In [12]:
fst ("x",2)
snd ("x",2)

"x"

2

In [13]:
map fst [("x",2), ("y",4)]

["x","y"]

In [14]:
-- "y"가 ["b","x","y"] 있는지 없는지

elem 2 [1,2,3]
elem 4 [1,2,3]

True

False

In [15]:
-- 환경 ["b","x","y"]가   ["x","y"]를 포함하나?

elem "x" ["b","x","y"]
elem "y" ["b","x","y"]

elem "x" ["b","x","y"] && elem "y" ["b","x","y"]

True

True

True

In [16]:
-- 환경 ["b","x","y"]가   ["x","y","z"]를 포함하나?
elem "x" ["b","x","y"]
elem "y" ["b","x","y"]
elem "z" ["b","x","y"]

elem "x" ["b","x","y"] && elem "y" ["b","x","y"] && elem "z" ["b","x","y"]

True

True

False

False

In [31]:
-- f [b1,b2,b3,...] = b1 && b2 && b3 && ...
f :: [Bool] -> Bool
f []     = True
f (b:bs) = b && f bs

f [True,True,True] 
f [True,False,True]

and [True,True,True]
and [True,False,True]

-- and = foldr (&&) True 와 같다

True

False

True

False

In [39]:
-- g [b1,b2,b3,...] = b1 || b2 || b3 || ...
g :: [Bool] -> Bool
g []     = False
g (b:bs) = b || g bs

g [False,False,False]
g [False,True,False]

or [False,False,False]
or [False,True,False]

-- or = foldr (||) False 와 같다

False

True

False

True